In [1]:
# ============================================================
# CELL 1: IMPORTS
# ============================================================

# ============================================================
# CORE NUMERICAL & DATA HANDLING
# ============================================================
import numpy as np
import pandas as pd

# ============================================================
# VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================
# STATISTICAL TESTING
# ============================================================
from scipy import stats

# ============================================================
# SKLEARN: MODEL SELECTION & VALIDATION
# ============================================================
from sklearn.model_selection import (
    StratifiedKFold,
    train_test_split
)

# ============================================================
# SKLEARN: MODELS
# ============================================================
from sklearn.ensemble import RandomForestClassifier

# ============================================================
# SKLEARN: METRICS & FEATURE IMPORTANCE
# ============================================================
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    mutual_info_score
)
from sklearn.inspection import permutation_importance

# ============================================================
# GRADIENT BOOSTING
# ============================================================
import xgboost as xgb

# ============================================================
# RULE / ILP / STRUCTURAL UTILITIES
# ============================================================
import itertools
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Dict, Tuple, Set

# ============================================================
# WARNINGS CONTROL
# ============================================================
import warnings
warnings.filterwarnings("ignore")


In [2]:
# ============================================================
# CELL 2: LOAD & INSPECT DATA
# ============================================================

# Load dataset
path = "/content/sample_data/bank-full.csv"
df = pd.read_csv(path, sep=";")

print(f"Shape: {df.shape}")
df.head()


Shape: (45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
# ============================================================
# CELL 3: CHECK INPUT TYPES
# ============================================================
print("📌 COLUMN SCHEMA\n" + "-"*60)
schema = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.astype(str),
    "n_unique": [df[c].nunique() for c in df.columns]
})
schema


📌 COLUMN SCHEMA
------------------------------------------------------------


,column,dtype,n_unique
age,age,int64,77
job,job,object,12
marital,marital,object,3
education,education,object,4
default,default,object,2
balance,balance,int64,7168
housing,housing,object,2
loan,loan,object,2
contact,contact,object,3
day,day,int64,31


In [4]:
# ============================================================
# CELL 5: EXAMINE CATAGORICAL COLUMNS
# ============================================================
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("📌 CATEGORICAL COLUMNS\n" + "-"*60)
categorical_cols

for col in categorical_cols:
    print(f"\n🔎 Column: {col}")
    print("-" * 40)

    values = (
        df[col]
        .astype(str)
        .value_counts(dropna=False)
        .sort_index()
    )

    print(values)


📌 CATEGORICAL COLUMNS
------------------------------------------------------------

🔎 Column: job
----------------------------------------
job
admin.           5171
blue-collar      9732
entrepreneur     1487
housemaid        1240
management       9458
retired          2264
self-employed    1579
services         4154
student           938
technician       7597
unemployed       1303
unknown           288
Name: count, dtype: int64

🔎 Column: marital
----------------------------------------
marital
divorced     5207
married     27214
single      12790
Name: count, dtype: int64

🔎 Column: education
----------------------------------------
education
primary       6851
secondary    23202
tertiary     13301
unknown       1857
Name: count, dtype: int64

🔎 Column: default
----------------------------------------
default
no     44396
yes      815
Name: count, dtype: int64

🔎 Column: housing
----------------------------------------
housing
no     20081
yes    25130
Name: count, dtype: int64

🔎 Co

In [5]:
# ============================================================
# CELL 6: PREPROCESSING - CONTACT AS ORDINAL
# ============================================================

print("\n" + "="*80)
print("🔧 CLEAN PREPROCESSING FROM SCRATCH (NUMERIC-ONLY)")
print("="*80)

print(f"\nOriginal data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# --------------------------------------------------
# 1. CREATE CLEAN df_proc FRAME
# --------------------------------------------------
print("\n" + "="*40)
print("1. CREATING CLEAN df_proc")
print("="*40)

df_proc = df.copy()

print(f"\nInitial shape: {df_proc.shape}")
print(f"Columns: {list(df_proc.columns)}")

# --------------------------------------------------
# 1.1 Target encoding (STRICT)
# --------------------------------------------------
print(f"\n🔍 Target column 'y' before conversion:")
print(df_proc['y'].value_counts())

df_proc['y'] = df_proc['y'].map({'yes': 1, 'no': 0})
assert df_proc['y'].notna().all(), "❌ Unexpected values in target y"
df_proc['y'] = df_proc['y'].astype('int8')

print(f"\n✅ Target 'y' converted:")
print(df_proc['y'].value_counts())

# --------------------------------------------------
# 1.2 Binary categorical encoding
# --------------------------------------------------
binary_cols = ['default', 'housing', 'loan']
binary_map = {'no': 0, 'yes': 1}

print(f"\n🔍 Binary columns before conversion:")
for col in binary_cols:
    print(f"{col}:\n{df_proc[col].value_counts()}")

for col in binary_cols:
    df_proc[col] = df_proc[col].map(binary_map)
    assert df_proc[col].notna().all(), f"❌ Unexpected values in {col}"
    df_proc[col] = df_proc[col].astype('int8')

print(f"\n✅ Binary columns converted:")
for col in binary_cols:
    print(f"{col}: {df_proc[col].unique()}")

# --------------------------------------------------
# 1.3 Month → ordinal (DROP RAW MONTH)
# --------------------------------------------------
print(f"\n🔍 Month column before conversion:")
print(df_proc['month'].value_counts().sort_index())

month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

df_proc['month_ordinal'] = df_proc['month'].map(month_map)
assert df_proc['month_ordinal'].notna().all(), "❌ Unexpected month values"
df_proc['month_ordinal'] = df_proc['month_ordinal'].astype('int8')

# DROP raw month (no objects allowed)
df_proc.drop(columns=['month'], inplace=True)

print(f"\n✅ Month converted to ordinal and raw month dropped")

# --------------------------------------------------
# 1.4 POUTCOME - EXPLICIT MAPPING
# --------------------------------------------------
print(f"\n🔍 poutcome column before conversion:")
print(df_proc['poutcome'].value_counts())

poutcome_map = {
    'unknown': 0,
    'failure': 1,
    'other': 2,
    'success': 3
}

df_proc['poutcome'] = df_proc['poutcome'].map(poutcome_map)
assert df_proc['poutcome'].notna().all(), "❌ Unexpected poutcome values"
df_proc['poutcome'] = df_proc['poutcome'].astype('int8')

print(f"\n✅ poutcome converted:")
print(df_proc['poutcome'].value_counts())

# --------------------------------------------------
# 1.5 CONTACT - ORDINAL MAPPING (NOT BINARY)
# --------------------------------------------------
print(f"\n🔍 contact column before conversion:")
print(df_proc['contact'].value_counts())

# Map contact as ORDINAL: cellular=0, telephone=1, unknown=2
contact_map = {
    'cellular': 0,
    'telephone': 1,
    'unknown': 2
}

df_proc['contact'] = df_proc['contact'].map(contact_map)
assert df_proc['contact'].notna().all(), "❌ Unexpected contact values"
df_proc['contact'] = df_proc['contact'].astype('int8')

print(f"\n✅ contact converted to ordinal (0=cellular, 1=telephone, 2=unknown):")
print(df_proc['contact'].value_counts().sort_index())

# --------------------------------------------------
# 1.6 Ordinal / Label encoding REMAINING categoricals
# --------------------------------------------------
categorical_cols = df_proc.select_dtypes(include=['object']).columns.tolist()

print(f"\n🔍 Ordinal-encoding remaining categoricals:")
print(categorical_cols)

for col in categorical_cols:
    df_proc[col] = df_proc[col].astype('category').cat.codes
    df_proc[col] = df_proc[col].astype('int16')

# --------------------------------------------------
# 1.7 HARD TYPE GUARANTEE
# --------------------------------------------------
non_numeric = df_proc.select_dtypes(exclude=['number']).columns.tolist()
assert len(non_numeric) == 0, f"❌ NON-NUMERIC COLUMNS REMAIN: {non_numeric}"

print("\n✅ ALL COLUMNS ARE NUMERIC")

# --------------------------------------------------
# 1.8 Sanity checks
# --------------------------------------------------
print(f"\n📊 Data types in df_proc:")
for dtype in df_proc.dtypes.unique():
    cols = df_proc.columns[df_proc.dtypes == dtype].tolist()
    print(f"{dtype}: {cols}")

print(f"\n🔍 Checking for null values:")
assert df_proc.isnull().sum().sum() == 0, "❌ Nulls detected"
print("✅ No null values")

print(f"\n🔍 Checking for duplicates:")
dup_count = df_proc.duplicated().sum()
print(f"Exact duplicate rows: {dup_count} ({dup_count/len(df_proc)*100:.2f}%)")

# --------------------------------------------------
# 1.9 Final confirmation
# --------------------------------------------------
print(f"\n✅ df_proc created successfully!")
print(f"Shape: {df_proc.shape}")
print(f"Memory usage: {df_proc.memory_usage().sum() / 1024**2:.2f} MB")


🔧 CLEAN PREPROCESSING FROM SCRATCH (NUMERIC-ONLY)

Original data shape: (45211, 17)
Columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

1. CREATING CLEAN df_proc

Initial shape: (45211, 17)
Columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

🔍 Target column 'y' before conversion:
y
no     39922
yes     5289
Name: count, dtype: int64

✅ Target 'y' converted:
y
0    39922
1     5289
Name: count, dtype: int64

🔍 Binary columns before conversion:
default:
default
no     44396
yes      815
Name: count, dtype: int64
housing:
housing
yes    25130
no     20081
Name: count, dtype: int64
loan:
loan
no     37967
yes     7244
Name: count, dtype: int64

✅ Binary columns converted:
default: [0 1]
housing: [1 0]
loan: [0 1]

🔍 Month column before con

In [6]:
"""
IMPORTANT NOTE ON DATA DISTRIBUTION & OUTLIERS:

We deliberately do NOT scale or alter the natural distributions of the data at this stage.
This includes not removing outliers, not applying normalization, and not transforming variables.

RATIONALE:
1. The observed skewness and extreme values represent real-world customer behavior where
   outliers often encode meaningful business signals rather than noise.

2. Overlapping spikes and distribution discontinuities may reflect:
   - Behavioral threshold effects (e.g., call duration regimes)
   - Temporal or seasonal campaign dynamics
   - Distinct customer segments with non-linear response patterns

3. Our feature engineering strategy (state-based binning and thresholds) is designed to
   capture these effects explicitly while preserving semantic interpretability.

4. The GLASS-BRW framework relies on discovering natural decision boundaries from data,
   rather than imposing artificial smoothness through scaling or normalization.

MODEL-SPECIFIC NOTE:
This choice is intentional for rule-based, threshold-driven models.
Scaling and outlier handling may be revisited for distance-based or linear models,
but are avoided here to preserve interpretability and behavioral signal fidelity.

NEXT STEPS:
- Evaluate whether specific extreme values degrade rule stability
- Apply targeted transformations only if they improve interpretability or robustness
- Continue prioritizing temporal and behavioral signal discovery over distribution smoothing

This approach reflects business reality, where extreme values often identify the most
valuable, risky, or responsive customer segments.
"""

'\nIMPORTANT NOTE ON DATA DISTRIBUTION & OUTLIERS:\n\nWe deliberately do NOT scale or alter the natural distributions of the data at this stage.\nThis includes not removing outliers, not applying normalization, and not transforming variables.\n\nRATIONALE:\n1. The observed skewness and extreme values represent real-world customer behavior where\n   outliers often encode meaningful business signals rather than noise.\n\n2. Overlapping spikes and distribution discontinuities may reflect:\n   - Behavioral threshold effects (e.g., call duration regimes)\n   - Temporal or seasonal campaign dynamics\n   - Distinct customer segments with non-linear response patterns\n\n3. Our feature engineering strategy (state-based binning and thresholds) is designed to\n   capture these effects explicitly while preserving semantic interpretability.\n\n4. The GLASS-BRW framework relies on discovering natural decision boundaries from data,\n   rather than imposing artificial smoothness through scaling or nor

In [7]:
# ============================================================
# CELL 11: GLASS-BRW STATE-BASED FEATURE ENGINEERING
# ============================================================

def engineer_features_bank(df_proc: pd.DataFrame) -> pd.DataFrame:
    """
    State-based feature engineering for GLASS-BRW.
    INPUT  : df_proc (FULLY NUMERIC preprocessed frame)
    OUTPUT : df_eng  (binary engineered features ONLY + target y)
    """

    print("🔍 Engineering state-based features for GLASS-BRW...")

    df_eng = pd.DataFrame(index=df_proc.index)
    df_eng["y"] = df_proc["y"].astype(np.int8)

    # =========================================================
    # TIER 1 — BEHAVIORAL
    # =========================================================
    duration = pd.to_numeric(df_proc["duration"], errors="coerce").fillna(0)
    duration_bin = pd.cut(
        duration,
        bins=[-1, 60, 180, 360, 600, np.inf],
        labels=["very_short", "short", "medium", "long", "very_long"]
    )
    for label in ["very_short", "short", "medium", "long", "very_long"]:
        df_eng[f"duration_{label}"] = (duration_bin == label).astype(np.int8)

    campaign = pd.to_numeric(df_proc["campaign"], errors="coerce").fillna(1)
    campaign = np.clip(campaign, 1, campaign.quantile(0.98))
    df_eng["campaign_single"]   = (campaign == 1).astype(np.int8)
    df_eng["campaign_light"]    = ((campaign >= 2) & (campaign <= 3)).astype(np.int8)
    df_eng["campaign_moderate"] = ((campaign >= 4) & (campaign <= 6)).astype(np.int8)
    df_eng["campaign_heavy"]    = (campaign > 6).astype(np.int8)

    previous = pd.to_numeric(df_proc["previous"], errors="coerce").fillna(0)
    df_eng["previous_first_time"]   = (previous == 0).astype(np.int8)
    df_eng["previous_light_repeat"] = ((previous >= 1) & (previous <= 2)).astype(np.int8)
    df_eng["previous_heavy_repeat"] = (previous >= 3).astype(np.int8)

    # poutcome: unknown=0, failure=1, other=2, success=3
    poutcome = df_proc["poutcome"].astype(np.int8)
    df_eng["poutcome_success"] = (poutcome == 3).astype(np.int8)
    df_eng["poutcome_failure"] = (poutcome == 1).astype(np.int8)
    df_eng["poutcome_other"]   = (poutcome == 2).astype(np.int8)
    df_eng["poutcome_unknown"] = (poutcome == 0).astype(np.int8)

    # =========================================================
    # TIER 2 — TEMPORAL / MEMORY
    # =========================================================
    m = df_proc["month_ordinal"].astype(np.int8)
    df_eng["month_spring_lift"]      = m.isin([2, 3, 4]).astype(np.int8)
    df_eng["month_fall_lift"]        = m.isin([9, 10, 11]).astype(np.int8)
    df_eng["month_december_special"] = (m == 12).astype(np.int8)
    df_eng["month_may_volatile"]     = (m == 5).astype(np.int8)
    df_eng["month_summer_dead"]      = m.isin([6, 7, 8]).astype(np.int8)
    df_eng["month_jan_low"]          = (m == 1).astype(np.int8)

    day = pd.to_numeric(df_proc["day"], errors="coerce").fillna(15)
    df_eng["day_early"] = (day <= 10).astype(np.int8)
    df_eng["day_mid"]   = ((day > 10) & (day <= 20)).astype(np.int8)
    df_eng["day_late"]  = (day > 20).astype(np.int8)

    pdays = pd.to_numeric(df_proc["pdays"], errors="coerce").fillna(-1)
    df_eng["pdays_never_contacted"] = (pdays == -1).astype(np.int8)
    df_eng["pdays_recent_contact"]  = ((pdays >= 0) & (pdays <= 30)).astype(np.int8)
    df_eng["pdays_mid_recency"]     = ((pdays > 30) & (pdays <= 180)).astype(np.int8)
    df_eng["pdays_stale_contact"]   = (pdays > 180).astype(np.int8)

    # =========================================================
    # TIER 3 — CONTEXTUAL
    # =========================================================
    age = pd.to_numeric(df_proc["age"], errors="coerce").fillna(40)
    df_eng["age_young"]         = (age < 30).astype(np.int8)
    df_eng["age_prime_working"] = ((age >= 30) & (age <= 45)).astype(np.int8)
    df_eng["age_mature_stable"] = ((age > 45) & (age <= 60)).astype(np.int8)
    df_eng["age_senior"]        = (age > 60).astype(np.int8)

    balance = pd.to_numeric(df_proc["balance"], errors="coerce").fillna(0)
    balance = np.clip(balance, balance.quantile(0.01), balance.quantile(0.99))
    df_eng["balance_zero_or_negative"] = (balance <= 0).astype(np.int8)
    df_eng["balance_low"]              = ((balance > 0) & (balance <= 500)).astype(np.int8)
    df_eng["balance_mid"]              = ((balance > 500) & (balance <= 5000)).astype(np.int8)
    df_eng["balance_high"]             = (balance > 5000).astype(np.int8)

    # So contact_not_cellular is just (contact == 1)
    contact = df_proc["contact"].astype(np.int8)
    df_eng["contact_not_cellular"] = (contact == 1).astype(np.int8)

    # housing/loan/default already binary int8 in df_proc
    for col in ["housing", "loan", "default"]:
        df_eng[col] = df_proc[col].astype(np.int8)

    # =========================================================
    # TIER 4 — HIGH-PRECISION CONDITIONAL SUBSCRIBE FLAGS
    # Based on empirical triple-condition analysis
    # =========================================================

    # Base condition: successful previous outcome
    success = df_eng["poutcome_success"] == 1

    # Duration regimes identified as high-signal
    dur_medium = df_eng["duration_medium"] == 1
    dur_long   = df_eng["duration_long"] == 1

    base_zone = success & (dur_medium | dur_long)

    # ---- Conditional flags (all binary, interpretable) ----
    df_eng["flag_success_medlong_housing"] = (
        base_zone & (df_eng["housing"] == 1)
    ).astype(np.int8)

    df_eng["flag_success_medlong_job"] = (
        base_zone & (df_proc["job"] >= 0)  # ordinal job bucket already encoded
    ).astype(np.int8)

    df_eng["flag_success_medlong_education"] = (
        base_zone & (df_proc["education"] >= 0)
    ).astype(np.int8)

    df_eng["flag_success_medlong_marital"] = (
        base_zone & (df_proc["marital"] >= 0)
    ).astype(np.int8)

    df_eng["flag_success_medlong_loan"] = (
        base_zone & (df_eng["loan"] == 0)
    ).astype(np.int8)

    df_eng["flag_success_medlong_default"] = (
        base_zone & (df_eng["default"] == 0)
    ).astype(np.int8)

    df_eng["flag_success_medlong_contact_not_cell"] = (
        base_zone & (df_eng["contact_not_cellular"] == 1)
    ).astype(np.int8)

    df_eng["flag_success_medlong_month_late"] = (
        base_zone & df_eng["month_fall_lift"]
    ).astype(np.int8)


    # --------------------------------------------------
    # FINAL: verify binary-only
    # --------------------------------------------------
    non_binary = []
    for col in df_eng.columns:
        if col == "y":
            continue
        u = pd.unique(df_eng[col])
        if not set(u).issubset({0, 1}):
            non_binary.append((col, u))

    if non_binary:
        print(f"❌ Found non-binary features: {len(non_binary)}")
        for c, u in non_binary[:10]:
            print(f"   {c}: {u}")
    else:
        print("✓ All engineered features are binary 0/1")

    print(f"✅ Engineered {df_eng.shape[1]-1} binary features (+ y)")
    return df_eng

In [8]:
# ============================================================
# CELL 12: RULE DATA STRUCTURE
# ============================================================

@dataclass
class Rule:

    rule_id: int
    segment: Set[Tuple[str, str]]
    predicted_class: int  # 0 = NOT_SUBSCRIBE, 1 = SUBSCRIBE
    complexity: int

    # Metrics computed during validation
    precision: float = 0.0
    recall: float = 0.0
    coverage: float = 0.0
    stability: float = 0.0
    interpretability: float = 0.0
    boundary_ambiguity: float = 0.0
    ebm_overlap: float = 0.0

    def __hash__(self):
        return hash((self.rule_id, frozenset(self.segment), self.predicted_class))

    def __eq__(self, other):
        return self.rule_id == other.rule_id

In [9]:
# ============================================================
# FIXED SEGMENT BUILDER - NO STRING CONVERSION
# ============================================================
class BankSegmentBuilder:
    # =========================================================
    # TIER 1 — BEHAVIORAL (direct interaction / outcome signals)
    # =========================================================
    TIER1_BEHAVIORAL = [
        "duration_very_long",
        "duration_short",
        "duration_long",
        "duration_medium",
        "campaign_single",
        "poutcome_success",

        # --- High-precision conditional behavioral flags ---
        "flag_success_medlong_housing",
        "flag_success_medlong_loan",
        "flag_success_medlong_default",
        "flag_success_medlong_contact_not_cell",
    ]

    # =========================================================
    # TIER 2 — TEMPORAL / MEMORY (when & cadence effects)
    # =========================================================
    TIER2_TEMPORAL = [
        "month_summer_dead",
        "month_spring_lift",
        "month_may_volatile",
        "day_late",
        "day_mid",

        # --- Conditional temporal amplification ---
        "flag_success_medlong_month_late",
    ]

    # =========================================================
    # TIER 3 — CONTEXTUAL (demographic / financial context)
    # =========================================================
    TIER3_CONTEXTUAL = [
        "balance_mid",
        "age_prime_working",
        "age_senior",
        "contact_not_cellular",

        # --- Conditional context flags ---
        "flag_success_medlong_job",
        "flag_success_medlong_education",
        "flag_success_medlong_marital",
    ]


    def __init__(self, mode="strict"):
        self.mode = mode

    def assign_segments(self, X):
        X = X.drop(columns=["y_bin"], errors="ignore")

        if self.mode == "strict":
            cols = self.TIER1_BEHAVIORAL + self.TIER2_TEMPORAL
        elif self.mode == "standard":
            cols = self.TIER1_BEHAVIORAL + self.TIER2_TEMPORAL + self.TIER3_CONTEXTUAL
        else:
            cols = self.TIER1_BEHAVIORAL + self.TIER2_TEMPORAL + self.TIER3_CONTEXTUAL

        # Filter to available columns
        cols = [c for c in cols if c in X.columns]

        missing_tier1 = [c for c in self.TIER1_BEHAVIORAL if c not in X.columns]
        if len(missing_tier1) > len(self.TIER1_BEHAVIORAL) * 0.5:
            raise ValueError(f"Too many missing Tier 1 features: {missing_tier1}")

        # Just select the columns - they're already 0/1
        segments = X[cols].copy()

        # Validate binary
        for col in segments.columns:
            vals = set(segments[col].dropna().unique())
            if not vals.issubset({0, 1}):
                raise ValueError(f"Non-binary feature: {col} -> {vals}")

        # Fill any NULLs with 0
        segments = segments.fillna(0).astype(int)

        return segments

    def validate_rule_composition(self, segment):
        feature_names = {feat.lower() for feat, _ in segment}
        tier1_names = {f.lower() for f in self.TIER1_BEHAVIORAL}
        has_tier1 = bool(feature_names & tier1_names)

        if not has_tier1:
            tier3_names = {f.lower() for f in self.TIER3_CONTEXTUAL}
            only_tier3 = feature_names.issubset(tier3_names)
            if only_tier3:
                return False

        return has_tier1 or len(feature_names & {f.lower() for f in self.TIER2_TEMPORAL}) > 0

In [10]:
# ============================================================
# CELL 14: RULE GENERATOR - FIXED FOR 0/1 VALUES
# ============================================================

class RuleGenerator:
    def __init__(self, min_support=6, max_complexity=3, min_complexity=1,
                 mode="strict", beam_width=300, min_precision_gain=0.0,
                 min_seed_precision_subscribe=0.70, min_seed_precision_not_subscribe=0.75,
                 min_recall_subscribe=0.05, min_recall_not_subscribe=0.25,
                 tier1_prefixes=("duration", "poutcome", "day",
                                 "age", "contact", "campaign",
                                 "flag", "poutcome", "balance")):
        self.min_support = min_support
        self.max_complexity = max_complexity
        self.min_complexity = min_complexity
        self.mode = mode
        self.beam_width = beam_width
        self.min_precision_gain = min_precision_gain
        self.min_seed_precision_subscribe = min_seed_precision_subscribe
        self.min_seed_precision_not_subscribe = min_seed_precision_not_subscribe
        self.min_recall_subscribe = min_recall_subscribe
        self.min_recall_not_subscribe = min_recall_not_subscribe
        self.tier1_prefixes = tier1_prefixes

    def _compute_rule_mask(self, segment, segments_df):
        mask = pd.Series(True, index=segments_df.index)
        for f, l in segment:
            mask &= (segments_df[f] == l)
        return mask

    def _compute_precision(self, mask, y, cls):
        if mask.sum() == 0:
            return 0.0
        return (y[mask] == cls).mean()

    def _compute_recall(self, mask, y, cls):
        total_class = (y == cls).sum()
        if total_class == 0:
            return 0.0
        true_positives = ((y == cls) & mask).sum()
        return true_positives / total_class

    def _has_tier1_feature(self, segment):
        feats = {f for f, _ in segment}
        return any(any(f.startswith(p) for p in self.tier1_prefixes) for f in feats)

    def _is_valid_semantic(self, segment):
        return self._has_tier1_feature(segment)

    def _score(self, p, r, c):
        return (p ** 2) * r * c

    def generate_candidates(self, segments_df, y):
        features = list(segments_df.columns)
        N = len(segments_df)

        print(f"Using {len(features)} binary features")
        print(f"Beam width: {self.beam_width}")
        print(f"Rule complexity: {self.min_complexity}–{self.max_complexity}")

        rule_id = 0
        all_candidates = []
        current = {0: [], 1: []}

        for f in features:
            for l in (1, 0):  # FIXED: Use 1, 0 instead of "yes", "no"
                seg = {(f, l)}
                if not self._is_valid_semantic(seg):
                    continue

                mask = self._compute_rule_mask(seg, segments_df)
                supp = mask.sum()
                if supp < self.min_support:
                    continue

                cov = supp / N

                for cls in (0, 1):
                    p = self._compute_precision(mask, y, cls)
                    r = self._compute_recall(mask, y, cls)

                    if cls == 1 and p < self.min_seed_precision_subscribe:
                        continue
                    if cls == 0 and p < self.min_seed_precision_not_subscribe:
                        continue
                    if cls == 1 and r < self.min_recall_subscribe:
                        continue
                    if cls == 0 and r < self.min_recall_not_subscribe:
                        continue

                    rule = Rule(rule_id=rule_id, segment=seg, predicted_class=cls,
                               complexity=1, interpretability=1.0)
                    rule._p, rule._r, rule._c = p, r, cov
                    rule._s = self._score(p, r, cov)
                    current[cls].append(rule)
                    rule_id += 1

        for cls in (0, 1):
            current[cls] = sorted(current[cls], key=lambda r: r._s, reverse=True)[:self.beam_width]

        print(f"[DEPTH 1] Seeds after beam prune: SUB={len(current[1])}, NOT_SUB={len(current[0])}")

        for depth in range(2, self.max_complexity + 1):
            next_rules = {0: [], 1: []}
            expansions = 0

            for cls in (0, 1):
                for parent in current[cls]:
                    used = {f for f, _ in parent.segment}

                    for f in features:
                        if f in used:
                            continue

                        for l in (1, 0):  # FIXED: Use 1, 0 instead of "yes", "no"
                            seg = parent.segment | {(f, l)}
                            if not self._is_valid_semantic(seg):
                                continue

                            mask = self._compute_rule_mask(seg, segments_df)
                            supp = mask.sum()
                            if supp < self.min_support:
                                continue

                            p = self._compute_precision(mask, y, cls)
                            r = self._compute_recall(mask, y, cls)

                            if p < parent._p - self.min_precision_gain:
                                continue
                            if cls == 1 and r < self.min_recall_subscribe:
                                continue
                            if cls == 0 and r < self.min_recall_not_subscribe:
                                continue

                            cov = supp / N
                            rule = Rule(rule_id=rule_id, segment=seg, predicted_class=cls,
                                       complexity=depth, interpretability=1.0 / depth)
                            rule._p, rule._r, rule._c = p, r, cov
                            rule._s = self._score(p, r, cov)
                            next_rules[cls].append(rule)
                            rule_id += 1
                            expansions += 1

            if expansions == 0:
                print(f"[DEPTH {depth}] No valid expansions — stopping early.")
                break

            for cls in (0, 1):
                next_rules[cls] = sorted(next_rules[cls], key=lambda r: r._s, reverse=True)[:self.beam_width]
                if depth >= self.min_complexity:
                    all_candidates.extend(next_rules[cls])

            print(f"[DEPTH {depth}] After beam prune: SUB={len(next_rules[1])}, NOT_SUB={len(next_rules[0])}")
            current = next_rules

        print(f"\n✅ Generated {len(all_candidates)} candidate rules (k ≥ {self.min_complexity})")
        return all_candidates

In [11]:
# ============================================================
# CELL 15: RULE EVALUATOR
# ============================================================
class RuleEvaluator:
    """
    Compute validation-based quality metrics for candidate rules.
    """

    def __init__(self, segment_builder, min_support=6):
        self.segment_builder = segment_builder
        self.min_support = min_support

    def match_rule(self, rule, segments_df):
        mask = pd.Series(True, index=segments_df.index)
        for feature, level in rule.segment:
            if feature in segments_df.columns:
                mask &= (segments_df[feature] == level)
            else:
                return pd.Series(False, index=segments_df.index)
        return mask

    def evaluate_candidates(self, candidates, X_val, y_val, ebm_model=None):
        segments_val = self.segment_builder.assign_segments(X_val)

        ebm_proba = None
        if ebm_model is not None:
            ebm_proba = ebm_model.predict_proba(X_val)[:, 1]

        evaluated_rules = []

        for rule in candidates:
            mask = self.match_rule(rule, segments_val)
            n_matches = mask.sum()
            rule.covered_idx = set(np.flatnonzero(mask.values))

            if n_matches < self.min_support // 2:
                continue

            y_pred = np.full(n_matches, rule.predicted_class)
            y_true = y_val[mask]

            # ✅ CLASS-SPECIFIC PRECISION
            if rule.predicted_class == 1:
                rule.precision = precision_score(y_true, y_pred, zero_division=0.0)
            else:
                rule.precision = (y_true == 0).sum() / len(y_true) if len(y_true) > 0 else 0.0

            # ✅ CLASS-SPECIFIC RECALL
            if rule.predicted_class == 1:
                total_positives = (y_val == 1).sum()
                true_positives = ((y_val == 1) & mask).sum()
                rule.recall = true_positives / total_positives if total_positives > 0 else 0.0
            else:
                total_negatives = (y_val == 0).sum()
                true_negatives = ((y_val == 0) & mask).sum()
                rule.recall = true_negatives / total_negatives if total_negatives > 0 else 0.0

            rule.coverage = n_matches / len(X_val)

            if rule.predicted_class == 1:
                rule.stability = self._estimate_stability(rule, X_val, y_val)
            else:
                rule.stability = 1.0

            if ebm_proba is not None:
                ebm_conf_in_segment = np.abs(ebm_proba[mask] - 0.5)
                rule.boundary_ambiguity = ebm_conf_in_segment.mean()
                rule.ebm_overlap = (ebm_conf_in_segment > 0.20).mean()
            else:
                rule.boundary_ambiguity = 0.5
                rule.ebm_overlap = 0.0

            evaluated_rules.append(rule)

        print(f"Evaluated {len(evaluated_rules)} rules "
              f"(filtered {len(candidates) - len(evaluated_rules)} low-coverage)")

        return evaluated_rules

    def _estimate_stability(self, rule, X, y, n_bootstrap=3):
        precisions = []
        n = len(X)

        for _ in range(n_bootstrap):
            idx = np.random.choice(n, size=n, replace=True)
            X_sample = X.iloc[idx]
            y_sample = y.iloc[idx] if hasattr(y, 'iloc') else y[idx]

            segments = self.segment_builder.assign_segments(X_sample)
            mask = self.match_rule(rule, segments)

            if mask.sum() < 10:
                continue

            y_pred = np.full(mask.sum(), rule.predicted_class)
            y_true = y_sample[mask]

            if rule.predicted_class == 1:
                prec = precision_score(y_true, y_pred, zero_division=0.0)
            else:
                prec = (y_true == 0).sum() / len(y_true) if len(y_true) > 0 else 0.0

            precisions.append(prec)

        if len(precisions) < 2:
            return 0.0

        mean_prec = np.mean(precisions)
        std_prec = np.std(precisions)

        if mean_prec > 0:
            stability = 1.0 - (std_prec / mean_prec)
            return max(0.0, stability)
        else:
            return 0.0

In [12]:
# ============================================================
# CELL 16: ILP SELECTOR - WITH FULL DEBUG
# ============================================================

class ILPRuleSelector:
    def __init__(self, min_rules=5, max_rules=10,
                 min_precision_subscribe=0.75, min_precision_not_subscribe=0.75,
                 min_recall_subscribe=0.05, min_recall_not_subscribe=0.25,
                 min_subscribe_rules=0, max_subscribe_rules=5,
                 min_not_subscribe_rules=0, max_not_subscribe_rules=5,
                 max_feature_usage=20,
                 lambda_boundary=0.10, lambda_overlap=0.25,
                 recall_weight=1.0):
        self.min_rules = min_rules
        self.max_rules = max_rules
        self.min_precision_subscribe = min_precision_subscribe
        self.min_precision_not_subscribe = min_precision_not_subscribe
        self.min_recall_subscribe = min_recall_subscribe
        self.min_recall_not_subscribe = min_recall_not_subscribe
        self.min_subscribe_rules = min_subscribe_rules
        self.max_subscribe_rules = max_subscribe_rules
        self.min_not_subscribe_rules = min_not_subscribe_rules
        self.max_not_subscribe_rules = max_not_subscribe_rules
        self.max_feature_usage = max_feature_usage
        self.lambda_boundary = lambda_boundary
        self.lambda_overlap = lambda_overlap
        self.recall_weight = recall_weight

    def select_rules(self, candidates):
        from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

        if not candidates:
            print("Warning: No candidates provided to ILP selector.")
            return []

        # ============================================================
        # DEBUG: Show ALL candidates before gating
        # ============================================================
        print("\n" + "="*80)
        print("🔍 ILP DEBUG: CANDIDATES BEFORE GATING")
        print("="*80)

        subscribe_cands = [r for r in candidates if r.predicted_class == 1]
        not_subscribe_cands = [r for r in candidates if r.predicted_class == 0]

        print(f"\nTotal candidates: {len(candidates)}")
        print(f"  SUBSCRIBE candidates: {len(subscribe_cands)}")
        print(f"  NOT_SUBSCRIBE candidates: {len(not_subscribe_cands)}")

        print(f"\n📊 Top 10 SUBSCRIBE candidates:")
        print(f"{'Rule':<6} {'Segment':<50} {'Precision':<10} {'Recall':<10} {'Coverage':<10}")
        print("-" * 90)
        for i, r in enumerate(sorted(subscribe_cands, key=lambda x: x.precision, reverse=True)[:10], 1):
            seg_str = ' AND '.join([f"{f}={l}" for f, l in list(r.segment)[:2]])
            if len(r.segment) > 2:
                seg_str += f" ... (+{len(r.segment)-2})"
            print(f"{i:<6} {seg_str:<50} {r.precision:<10.3f} {r.recall:<10.3f} {r.coverage:<10.3f}")

        print(f"\n📊 Top 10 NOT_SUBSCRIBE candidates:")
        print(f"{'Rule':<6} {'Segment':<50} {'Precision':<10} {'Recall':<10} {'Coverage':<10}")
        print("-" * 90)
        for i, r in enumerate(sorted(not_subscribe_cands, key=lambda x: x.precision, reverse=True)[:10], 1):
            seg_str = ' AND '.join([f"{f}={l}" for f, l in list(r.segment)[:2]])
            if len(r.segment) > 2:
                seg_str += f" ... (+{len(r.segment)-2})"
            print(f"{i:<6} {seg_str:<50} {r.precision:<10.3f} {r.recall:<10.3f} {r.coverage:<10.3f}")

        # ============================================================
        # Setup ILP
        # ============================================================
        max_seen = -1
        for r in candidates:
            if hasattr(r, "covered_idx") and r.covered_idx and len(r.covered_idx) > 0:
                max_seen = max(max_seen, max(r.covered_idx))

        N = max_seen + 1
        if N <= 0:
            print("Warning: Could not infer validation size.")
            return []

        prob = LpProblem("GLASS_BRW_Bank_Selection", LpMaximize)
        x = {r.rule_id: LpVariable(f"x_{r.rule_id}", cat='Binary') for r in candidates}

        # NO MORE z variables - we don't care about coverage!

        objective_terms = []
        for r in candidates:
            # Simple objective: precision^2 * recall * coverage
            quality = (r.precision ** 2) * r.recall * r.coverage * r.interpretability
            objective_terms.append(x[r.rule_id] * quality)

        prob += lpSum(objective_terms), "Objective"

        # ============================================================
        # DEBUG: Apply gates and show results
        # ============================================================
        print("\n" + "="*80)
        print("🔍 APPLYING PRECISION/RECALL GATES")
        print("="*80)
        print(f"\nGate thresholds:")
        print(f"  SUBSCRIBE:     precision ≥ {self.min_precision_subscribe:.3f}, recall ≥ {self.min_recall_subscribe:.3f}")
        print(f"  NOT_SUBSCRIBE: precision ≥ {self.min_precision_not_subscribe:.3f}, recall ≥ {self.min_recall_not_subscribe:.3f}")

        subscribe_valid = []
        subscribe_rejected = []
        not_subscribe_valid = []
        not_subscribe_rejected = []

        for r in candidates:
            if r.predicted_class == 1:
                if r.precision >= self.min_precision_subscribe and r.recall >= self.min_recall_subscribe:
                    subscribe_valid.append(r)
                else:
                    subscribe_rejected.append(r)
                    prob += (x[r.rule_id] == 0, f"Gate_Sub_{r.rule_id}")
            else:
                if r.precision >= self.min_precision_not_subscribe and r.recall >= self.min_recall_not_subscribe:
                    not_subscribe_valid.append(r)
                else:
                    not_subscribe_rejected.append(r)
                    prob += (x[r.rule_id] == 0, f"Gate_NotSub_{r.rule_id}")

        print(f"\n✅ PASSED GATES:")
        print(f"  SUBSCRIBE:     {len(subscribe_valid)}/{len(subscribe_cands)}")
        print(f"  NOT_SUBSCRIBE: {len(not_subscribe_valid)}/{len(not_subscribe_cands)}")

        print(f"\n❌ REJECTED BY GATES:")
        print(f"  SUBSCRIBE:     {len(subscribe_rejected)}/{len(subscribe_cands)}")
        print(f"  NOT_SUBSCRIBE: {len(not_subscribe_rejected)}/{len(not_subscribe_cands)}")

        # Show why SUBSCRIBE rules failed
        if len(subscribe_rejected) > 0:
            print(f"\n❌ Top 5 REJECTED SUBSCRIBE rules (showing why):")
            for i, r in enumerate(sorted(subscribe_rejected, key=lambda x: x.precision, reverse=True)[:5], 1):
                seg_str = ' AND '.join([f"{f}={l}" for f, l in list(r.segment)[:2]])
                if len(r.segment) > 2:
                    seg_str += f" ... (+{len(r.segment)-2})"

                fail_reasons = []
                if r.precision < self.min_precision_subscribe:
                    fail_reasons.append(f"precision {r.precision:.3f} < {self.min_precision_subscribe:.3f}")
                if r.recall < self.min_recall_subscribe:
                    fail_reasons.append(f"recall {r.recall:.3f} < {self.min_recall_subscribe:.3f}")

                print(f"  {i}. {seg_str}")
                print(f"     FAILED: {', '.join(fail_reasons)}")

        # Adjust minimums
        actual_min_sub = min(self.min_subscribe_rules, len(subscribe_valid))
        actual_min_not_sub = min(self.min_not_subscribe_rules, len(not_subscribe_valid))
        actual_min_rules = min(self.min_rules, len(subscribe_valid) + len(not_subscribe_valid))

        prob += (lpSum(x[r.rule_id] for r in candidates) >= actual_min_rules, "Min_Rules")
        prob += (lpSum(x[r.rule_id] for r in candidates) <= self.max_rules, "Max_Rules")

        if subscribe_valid:
            prob += (lpSum(x[r.rule_id] for r in subscribe_valid) >= actual_min_sub, "Min_Sub")
            prob += (lpSum(x[r.rule_id] for r in subscribe_valid) <= self.max_subscribe_rules, "Max_Sub")

        if not_subscribe_valid:
            prob += (lpSum(x[r.rule_id] for r in not_subscribe_valid) >= actual_min_not_sub, "Min_NotSub")
            prob += (lpSum(x[r.rule_id] for r in not_subscribe_valid) <= self.max_not_subscribe_rules, "Max_NotSub")

        # NO COVERAGE CONSTRAINT!

        feature_usage = defaultdict(list)
        for r in candidates:
            for feature, _ in r.segment:
                feature_usage[feature].append(r.rule_id)

        for feature, rule_ids in feature_usage.items():
            prob += (lpSum(x[rid] for rid in rule_ids) <= self.max_feature_usage, f"Div_{feature}")

        print("\n" + "="*80)
        print("Solving ILP (NO coverage constraint)...")
        solver = PULP_CBC_CMD(msg=0, timeLimit=300)
        prob.solve(solver)

        status = LpStatus[prob.status]
        print(f"ILP Status: {status}")
        print("="*80)

        if status != 'Optimal':
            print("\n⚠️ ILP FAILED - Using greedy fallback")

            subscribe_valid = sorted(subscribe_valid,
                                    key=lambda r: (r.precision ** 2) * r.recall * r.coverage,
                                    reverse=True)
            not_subscribe_valid = sorted(not_subscribe_valid,
                                        key=lambda r: (r.precision ** 2) * r.recall * r.coverage,
                                        reverse=True)

            selected_subscribe = subscribe_valid[:self.max_subscribe_rules]
            selected_not_subscribe = not_subscribe_valid[:self.max_not_subscribe_rules]

            selected_rules = selected_subscribe + selected_not_subscribe

            print(f"   Selected {len(selected_subscribe)} SUBSCRIBE rules")
            print(f"   Selected {len(selected_not_subscribe)} NOT_SUBSCRIBE rules")
            print(f"   Total: {len(selected_rules)} rules")

            return selected_rules

        selected_rules = []
        for r in candidates:
            if x[r.rule_id].varValue is not None and x[r.rule_id].varValue > 0.5:
                selected_rules.append(r)

        print(f"\nSelected {len(selected_rules)} rules")
        return selected_rules

In [13]:
# ============================================================
# CELL 17: GLASS-BRW CLASSIFIER (ADD RECALL TO PRINTING)
# ============================================================

class GLASS_BRW:
    def __init__(self,
                 mode="strict",
                 min_precision_subscribe=0.75,
                 min_precision_not_subscribe=0.75,
                 min_recall_subscribe=0.05,
                 min_recall_not_subscribe=0.25,
                 min_support=5,
                 max_complexity=3,
                 min_rules=8,
                 max_rules=10,
                 max_subscribe_rules=5,
                 max_not_subscribe_rules=5):

        """
        Args:
            mode: Segmentation mode ('strict', 'helpers', 'discover')
            min_precision_subscribe: Precision gate for SUBSCRIBE rules
            min_precision_not_subscribe: Precision gate for NOT_SUBSCRIBE rules
            min_support: Minimum samples for rule generation
            max_complexity: Maximum conditions per rule (k ≤ 3)
            min_rules: Minimum rules to select
            max_rules: Maximum rules to select
            max_subscribe_rules: Maximum SUBSCRIBE rules
            max_not_subscribe_rules: Maximum NOT_SUBSCRIBE rules
        """
        self.mode = mode
        self.min_precision_subscribe = min_precision_subscribe
        self.min_precision_not_subscribe = min_precision_not_subscribe
        self.min_support = min_support
        self.max_complexity = max_complexity
        self.min_rules = min_rules
        self.max_rules = max_rules
        self.max_subscribe_rules = max_subscribe_rules
        self.max_not_subscribe_rules = max_not_subscribe_rules

        # Components
        self.segment_builder = BankSegmentBuilder(mode=mode)
        self.rule_generator = RuleGenerator(min_support, max_complexity, mode=mode)
        self.rule_evaluator = RuleEvaluator(self.segment_builder, min_support)
        self.ilp_selector = ILPRuleSelector(
            min_rules=min_rules,
            max_rules=max_rules,
            min_precision_subscribe=min_precision_subscribe,
            min_precision_not_subscribe=min_precision_not_subscribe,
            min_subscribe_rules=3,
            max_subscribe_rules=max_subscribe_rules,
            min_not_subscribe_rules=3,
            max_not_subscribe_rules=max_not_subscribe_rules
        )

        # Fitted attributes
        self.rules = None
        self.is_fitted = False

    def fit(self, X_train, y_train, X_val, y_val, ebm_model=None):
        """
        Train GLASS-BRW using ILP optimization.

        Args:
            X_train: Training features (with engineered bank features)
            y_train: Training labels (0/1)
            X_val: Validation features
            y_val: Validation labels
            ebm_model: Optional EBM model for complementarity

        Returns:
            self
        """
        print("="*60)
        print("GLASS-BRW Training Pipeline (Bank Marketing)")
        print("="*60)
        print(f"Mode: {self.mode}")

        # Convert to pandas
        if not isinstance(X_train, pd.DataFrame):
            X_train = pd.DataFrame(X_train)
        if not isinstance(X_val, pd.DataFrame):
            X_val = pd.DataFrame(X_val)
        if not isinstance(y_train, (pd.Series, np.ndarray)):
            y_train = np.array(y_train)
        if not isinstance(y_val, (pd.Series, np.ndarray)):
            y_val = np.array(y_val)

        # Phase 1: Assign segments
        print("\nPhase 1: Assigning customer segments...")
        segments_train = self.segment_builder.assign_segments(X_train)
        print(f"  Segment features: {len(segments_train.columns)}")

        # Phase 2: Generate candidates
        print("\nPhase 2: Generating candidate rules...")
        candidates = self.rule_generator.generate_candidates(segments_train, y_train)

        # Phase 3: Evaluate on validation
        print("\nPhase 3: Evaluating rule quality on validation set...")
        evaluated_candidates = self.rule_evaluator.evaluate_candidates(
            candidates, X_val, y_val, ebm_model
        )

        # Phase 4: ILP selection
        print("\nPhase 4: Solving ILP for optimal rule subset...")
        selected_rules = self.ilp_selector.select_rules(evaluated_candidates)

        # Phase 5: Order rules (SUBSCRIBE first, precision-sorted)
        print("\nPhase 5: Ordering rules (SUBSCRIBE → NOT_SUBSCRIBE, precision-sorted)...")
        subscribe_rules = sorted(
            [r for r in selected_rules if r.predicted_class == 1],
            key=lambda r: r.precision,
            reverse=True
        )
        not_subscribe_rules = sorted(
            [r for r in selected_rules if r.predicted_class == 0],
            key=lambda r: r.precision,
            reverse=True
        )
        self.rules = subscribe_rules + not_subscribe_rules

        # ✅ UPDATED: Print final rules WITH RECALL
        print("\n" + "="*60)
        print("Selected Rules (ordered by precision):")
        print("="*60)
        for i, rule in enumerate(self.rules, 1):
            segment_str = ' AND '.join([f"{f}={l}" for f, l in rule.segment])
            class_str = "SUBSCRIBE" if rule.predicted_class == 1 else "NOT_SUB"
            print(f"{i}. [{class_str:9s}] {segment_str}")
            print(f"   Precision: {rule.precision:.1%}, Recall: {rule.recall:.1%}, "
                  f"Coverage: {rule.coverage:.1%}, Complexity: {rule.complexity}")
        print("="*60)

        self.is_fitted = True
        return self

    def predict(self, X):
        """
        Predict using first-match-wins with explicit abstention.

        Args:
            X: Features (DataFrame with engineered features)

        Returns:
            predictions: Array of {-1, 0, 1} where -1 = ABSTAIN
            explanations: List of explanations
            confidences: Array of rule precisions
        """
        if not self.is_fitted:
            raise ValueError("Model must be fitted before prediction")

        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        n = len(X)
        predictions = np.full(n, -1, dtype=int)
        explanations = [None] * n
        confidences = np.zeros(n)

        # Assign segments
        segments = self.segment_builder.assign_segments(X)

        # First-match-wins
        for i in range(n):
            for rule in self.rules:
                # Check segment match
                matches = True
                for feature, level in rule.segment:
                    if feature not in segments.columns:
                        matches = False
                        break
                    if segments.iloc[i][feature] != level:
                        matches = False
                        break

                if not matches:
                    continue

                # Optional: enforce precision floor at execution
                if rule.predicted_class == 1 and rule.precision < self.min_precision_subscribe:
                    continue
                if rule.predicted_class == 0 and rule.precision < self.min_precision_not_subscribe:
                    continue

                predictions[i] = rule.predicted_class
                confidences[i] = rule.precision
                explanations[i] = self._format_explanation(rule)
                break

            # No rule matched = abstain
            if predictions[i] == -1:
                explanations[i] = "ABSTAIN: No high-precision segment rule applies"
                confidences[i] = 0.0

        self.last_predictions = predictions
        self.last_explanations = explanations
        self.last_confidences = confidences

        return predictions, explanations, confidences

    def predict_proba(self, X):
        """
        Return probability estimates [P(Not_Subscribe), P(Subscribe)].

        Args:
            X: Features

        Returns:
            probas: Array of shape (n_samples, 2)
        """
        predictions, _, confidences = self.predict(X)
        n = len(X)
        probas = np.zeros((n, 2))

        for i in range(n):
            if predictions[i] == -1:  # ABSTAIN
                probas[i] = [0.5, 0.5]
            elif predictions[i] == 1:  # SUBSCRIBE
                probas[i] = [1 - confidences[i], confidences[i]]
            else:  # NOT_SUBSCRIBE
                probas[i] = [confidences[i], 1 - confidences[i]]

        return probas

    def _format_explanation(self, rule):
        """Generate human-readable explanation for a rule."""
        conditions = [f"{feat}={level}" for feat, level in rule.segment]
        segment_str = ' AND '.join(conditions)
        class_name = "SUBSCRIBE" if rule.predicted_class == 1 else "NOT_SUBSCRIBE"
        return f"Segment: {segment_str} → {class_name} ({rule.precision:.1%} precision)"

    def explain_prediction(self, X, sample_idx=0):
        """Get explanation for a specific prediction."""
        if not hasattr(self, 'last_explanations'):
            self.predict(X)
        if sample_idx < len(self.last_explanations):
            return self.last_explanations[sample_idx]
        return "No explanation available"

    def get_coverage_stats(self, X, y):
        """Compute coverage and performance statistics."""
        predictions, _, confidences = self.predict(X)

        covered = (predictions != -1)
        coverage_rate = covered.mean()
        abstention_rate = 1 - coverage_rate

        if covered.sum() > 0:
            covered_precision = precision_score(y[covered], predictions[covered])
            covered_recall = recall_score(y[covered], predictions[covered])
        else:
            covered_precision = 0.0
            covered_recall = 0.0

        subscribe_covered = covered & (predictions == 1)
        not_subscribe_covered = covered & (predictions == 0)

        stats = {
            'coverage_rate': coverage_rate,
            'abstention_rate': abstention_rate,
            'covered_precision': covered_precision,
            'covered_recall': covered_recall,
            'subscribe_coverage': subscribe_covered.mean(),
            'not_subscribe_coverage': not_subscribe_covered.mean(),
            'avg_confidence': confidences[covered].mean() if covered.sum() > 0 else 0.0,
            'n_samples': len(X),
            'n_covered': covered.sum(),
            'n_abstained': (~covered).sum()
        }

        return stats



In [14]:
# ============================================================
# CELL 18: GLASS-BRW MAIN PIPELINE
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. FEATURE ENGINEERING
# ============================================================
print("\n" + "="*80)
print("🔍 STEP 1: FEATURE ENGINEERING")
print("="*80)

df_eng = engineer_features_bank(df_proc)

top_15_features = [
    "duration_very_long", "duration_short", "duration_long", "duration_medium",
    "poutcome_success", "month_may_volatile", "age_prime_working", "age_senior",
    "contact_not_cellular", "campaign_single", "month_summer_dead", "month_spring_lift",
    "day_late", "day_mid", "balance_mid", "flag_success_medlong_job",
    "flag_success_medlong_education", "flag_success_medlong_month_late", "flag_success_medlong_marital",
    "flag_success_medlong_housing", "flag_success_medlong_loan",
    "flag_success_medlong_default", "flag_success_medlong_contact_not_cell"
]

available_features = [f for f in top_15_features if f in df_eng.columns]
missing_features = [f for f in top_15_features if f not in df_eng.columns]

print(f"\n✅ Available: {len(available_features)}/{len(top_15_features)}")
if missing_features:
    raise ValueError(f"❌ Missing features: {missing_features}")

y_clean = df_eng['y'].copy()
X_clean = df_eng[available_features].copy()

print(f"   Features: {X_clean.shape[1]}, Samples: {X_clean.shape[0]}")
print(f"   Positive rate: {y_clean.mean():.2%}")

# ============================================================
# 2. SAFE 70/15/15 SPLIT
# ============================================================
print("\n" + "="*80)
print("📊 STEP 2: 70/15/15 TRAIN/VAL/TEST SPLIT")
print("="*80)

X_temp, X_test, y_temp, y_test = train_test_split(
    X_clean, y_clean, test_size=0.15, random_state=42, stratify=y_clean
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.15/0.85, random_state=42, stratify=y_temp
)

print(f"\n✅ Split complete:")
print(f"   Train: {len(X_train)} ({len(X_train)/len(X_clean):.1%}) - pos rate: {y_train.mean():.2%}")
print(f"   Val:   {len(X_val)} ({len(X_val)/len(X_clean):.1%}) - pos rate: {y_val.mean():.2%}")
print(f"   Test:  {len(X_test)} ({len(X_test)/len(X_clean):.1%}) - pos rate: {y_test.mean():.2%}")

# ============================================================
# 3. TRAIN GLASS-BRW
# ============================================================
print("\n" + "="*80)
print("🚀 STEP 3: TRAINING GLASS-BRW")
print("="*80)

glass = GLASS_BRW(
    mode="helpers",
    min_precision_subscribe=0.70,
    min_precision_not_subscribe=0.50,
    min_recall_subscribe=0.05,
    min_recall_not_subscribe=0.10,
    min_support=5,
    max_complexity=3,
    min_rules=8,
    max_rules=10,
    max_subscribe_rules=5,
    max_not_subscribe_rules=5
)

glass.fit(X_train, y_train, X_val, y_val, ebm_model=None)

# ============================================================
# 4. EVALUATE ON TEST SET
# ============================================================
print("\n" + "="*80)
print("📊 STEP 4: EVALUATION ON TEST SET")
print("="*80)

predictions, explanations, confidences = glass.predict(X_test)
probas = glass.predict_proba(X_test)

covered = (predictions != -1)
preds_for_metrics = predictions.copy()
preds_for_metrics[preds_for_metrics == -1] = 0

print(f"\n📈 Coverage:")
print(f"   Covered:   {covered.sum():5d} ({covered.mean():.1%})")
print(f"   Abstained: {(~covered).sum():5d} ({(~covered).mean():.1%})")

print(f"\n📊 Overall Performance (abstain → NOT_SUBSCRIBE):")
print(f"   Accuracy:  {accuracy_score(y_test, preds_for_metrics):.3f}")
print(f"   Precision: {precision_score(y_test, preds_for_metrics, zero_division=0):.3f}")
print(f"   Recall:    {recall_score(y_test, preds_for_metrics, zero_division=0):.3f}")
print(f"   F1 Score:  {f1_score(y_test, preds_for_metrics, zero_division=0):.3f}")

if covered.sum() > 0:
    print(f"\n📊 Covered Cases Only:")
    print(f"   Accuracy:  {accuracy_score(y_test[covered], predictions[covered]):.3f}")
    print(f"   Precision: {precision_score(y_test[covered], predictions[covered], zero_division=0):.3f}")
    print(f"   Recall:    {recall_score(y_test[covered], predictions[covered], zero_division=0):.3f}")
    print(f"   F1 Score:  {f1_score(y_test[covered], predictions[covered], zero_division=0):.3f}")
    print(f"   Confidence: {confidences[covered].mean():.3f}")

    cm = confusion_matrix(y_test[covered], predictions[covered])
    print(f"\n📊 Confusion Matrix:")
    print(f"                Pred NOT_SUB  Pred SUBSCRIBE")
    print(f"Actual NOT_SUB      {cm[0,0]:5d}          {cm[0,1]:5d}")
    print(f"Actual SUBSCRIBE    {cm[1,0]:5d}          {cm[1,1]:5d}")

    print(f"\n📊 Classification Report:")
    print(classification_report(y_test[covered], predictions[covered],
                                target_names=['NOT_SUBSCRIBE', 'SUBSCRIBE'], zero_division=0))

# ============================================================
# 5. RULE ANALYSIS
# ============================================================
print("\n" + "="*80)
print("🔍 STEP 5: RULE ANALYSIS")
print("="*80)

subscribe_rules = [r for r in glass.rules if r.predicted_class == 1]
not_subscribe_rules = [r for r in glass.rules if r.predicted_class == 0]

print(f"\nTotal Rules: {len(glass.rules)}")
print(f"  SUBSCRIBE: {len(subscribe_rules)}")
print(f"  NOT_SUBSCRIBE: {len(not_subscribe_rules)}")

if subscribe_rules:
    print(f"\n🔍 Top 5 SUBSCRIBE Rules:")
    for i, r in enumerate(subscribe_rules[:5], 1):
        seg = ' AND '.join([f"{f}={l}" for f, l in r.segment])
        print(f"  {i}. {seg}")
        print(f"     P={r.precision:.1%}, R={r.recall:.1%}, Cov={r.coverage:.1%}, K={r.complexity}")

if not_subscribe_rules:
    print(f"\n🔍 Top 5 NOT_SUBSCRIBE Rules:")
    for i, r in enumerate(not_subscribe_rules[:5], 1):
        seg = ' AND '.join([f"{f}={l}" for f, l in r.segment])
        print(f"  {i}. {seg}")
        print(f"     P={r.precision:.1%}, R={r.recall:.1%}, Cov={r.coverage:.1%}, K={r.complexity}")

# Feature usage
feature_usage = {}
for r in glass.rules:
    for f, _ in r.segment:
        feature_usage[f] = feature_usage.get(f, 0) + 1

print(f"\n📊 Feature Usage:")
for f, cnt in sorted(feature_usage.items(), key=lambda x: x[1], reverse=True):
    print(f"   {f:25s}: {cnt:2d} ({cnt/len(glass.rules)*100:5.1f}%)")

unused = set(available_features) - set(feature_usage.keys())
if unused:
    print(f"\n⚠️  Unused: {unused}")

print("\n" + "="*80)
print("✅ COMPLETE")
print("="*80)


🔍 STEP 1: FEATURE ENGINEERING
🔍 Engineering state-based features for GLASS-BRW...
✓ All engineered features are binary 0/1
✅ Engineered 49 binary features (+ y)

✅ Available: 23/23
   Features: 23, Samples: 45211
   Positive rate: 11.70%

📊 STEP 2: 70/15/15 TRAIN/VAL/TEST SPLIT

✅ Split complete:
   Train: 31647 (70.0%) - pos rate: 11.70%
   Val:   6782 (15.0%) - pos rate: 11.69%
   Test:  6782 (15.0%) - pos rate: 11.69%

🚀 STEP 3: TRAINING GLASS-BRW
GLASS-BRW Training Pipeline (Bank Marketing)
Mode: helpers

Phase 1: Assigning customer segments...
  Segment features: 23

Phase 2: Generating candidate rules...
Using 23 binary features
Beam width: 300
Rule complexity: 1–3
[DEPTH 1] Seeds after beam prune: SUB=5, NOT_SUB=27
[DEPTH 2] After beam prune: SUB=80, NOT_SUB=300
[DEPTH 3] After beam prune: SUB=300, NOT_SUB=300

✅ Generated 980 candidate rules (k ≥ 1)

Phase 3: Evaluating rule quality on validation set...
Evaluated 980 rules (filtered 0 low-coverage)

Phase 4: Solving ILP for op